# Spider SFT Data Prep
Build instruction-tuning data (messages + text) from Spider train set.

In [1]:
import json
from pathlib import Path


In [2]:
DATA_DIR = Path('/home/geniusjjjjj/data/spider')
# TRAIN_JSON = DATA_DIR / 'train_spider.json'
TRAIN_JSON = DATA_DIR / 'dev.json'
TABLES_JSON = DATA_DIR / 'tables.json'

OUT_DIR = Path('/home/geniusjjjjj/transformer/SFT/data')
OUT_DIR.mkdir(parents=True, exist_ok=True)
OUT_JSONL = OUT_DIR / 'spider_dev_sft.jsonl'

# Optional: limit size for a quick smoke test
MAX_SAMPLES = None  # e.g. 1000


In [3]:
train_data = json.loads(TRAIN_JSON.read_text())
tables = {t['db_id']: t for t in json.loads(TABLES_JSON.read_text())}

def schema_to_text(db_id):
    t = tables[db_id]
    table_names = t['table_names_original']
    column_names = t['column_names_original']

    cols_by_table = {i: [] for i in range(len(table_names))}
    for tbl_idx, col in column_names:
        if tbl_idx == -1:
            continue
        cols_by_table[tbl_idx].append(col)

    parts = []
    for i, tbl in enumerate(table_names):
        cols = ', '.join(cols_by_table[i])
        parts.append(f"{tbl}({cols})")
    return ' ; '.join(parts)


In [4]:
def build_system_prompt() -> str:
    return (
        "You are a text-to-SQL system.\n"
        "Use ONLY tables/columns from the schema.\n"
        "Return exactly ONE SQLite SQL query and nothing else.\n"
        "Do NOT include explanations, comments, code fences, or the database id."
    )

def build_user_prompt(question: str, db_id: str) -> str:
    schema = schema_to_text(db_id)
    return (
        f"Database id: {db_id}\n"
        f"Schema: {schema}\n"
        f"Question: {question}\n"
        "SQL:"
    )

def build_example(ex):
    db_id = ex["db_id"]
    question = ex["question"]
    gold_sql = ex["query"]

    system = build_system_prompt()
    user = build_user_prompt(question, db_id)

    messages = [
        {"role": "system", "content": system},
        {"role": "user", "content": user},
        {"role": "assistant", "content": gold_sql},
    ]

    # Plain text fallback (kept consistent with system+user framing).
    # If you later decide to train from `text` instead of `messages`,
    # this will better match the chat-template distribution.
    text = system + "\n\n" + user + "\n" + gold_sql

    return {
        "db_id": db_id,
        "question": question,
        "gold_sql": gold_sql,
        "messages": messages,
        "text": text,
    }


In [5]:
# Build and save dataset
count = 0
with OUT_JSONL.open('w', encoding='utf-8') as f:
    for ex in train_data:
        if MAX_SAMPLES is not None and count >= MAX_SAMPLES:
            break
        item = build_example(ex)
        f.write(json.dumps(item, ensure_ascii=False) + '\n')
        count += 1

print('Wrote:', OUT_JSONL)
print('Samples:', count)


Wrote: /home/geniusjjjjj/transformer/SFT/data/spider_dev_sft.jsonl
Samples: 1034
